### Quick intro to LlamaIndex  
Sources: [1](https://lmy.medium.com/comparing-langchain-and-llamaindex-with-4-tasks-2970140edf33), [2](https://docs.llamaindex.ai/en/stable/), [3](https://github.com/run-llama/llama_index), [4](https://nanonets.com/blog/llamaindex/)  

LlamaIndex is a "data framework" to help you build LLM apps. It provides the following tools:

+ Offers data connectors to ingest your existing data sources and data formats (APIs, PDFs, docs, SQL, etc.).
+ Provides ways to structure your data (indices, graphs) so that this data can be easily used with LLMs.
+ Provides an advanced retrieval/query interface over your data: Feed in any LLM input prompt, get back retrieved context and knowledge-augmented output.
+ Allows easy integrations with your outer application framework (e.g. with LangChain, Flask, Docker, ChatGPT, anything else).
+ LlamaIndex provides tools for both beginner users and advanced users.  

The high-level API allows beginner users to use LlamaIndex to ingest and query their data in 5 lines of code.  
The lower-level APIs allow advanced users to customize and extend any module (data connectors, indices, retrievers, query engines, reranking modules), to fit their needs.  

LlamaIndex provides the following tools:
+ Data connectors ingest your existing data from their native source and format. These could be APIs, PDFs, SQL, and (much) more.
+ Data indexes structure your data in intermediate representations that are easy and performant for LLMs to consume.
+ Engines provide natural language access to your data. For example:
+ Query engines are powerful retrieval interfaces for knowledge-augmented output.
+ Chat engines are conversational interfaces for multi-message, “back and forth” interactions with your data.
+ Data agents are LLM-powered knowledge workers augmented by tools, from simple helper functions to API integrations and more.
+ Application integrations tie LlamaIndex back into the rest of your ecosystem. This could be LangChain, Flask, Docker, ChatGPT, or… anything else!  

#### Installing Packages


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install -qU openai
!pip install -qU llama-index
!pip install -qU pydantic
!pip install -qU llama-index-llms-openai
#%pip install llama-index-embeddings-ollama
!pip install -qU pypdf
!pip install -qU docx2txt
!pip install -qU prettyprint


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Importing Packages

In [2]:
import os
import sys
import openai
import pydantic
import llama_index

from llama_index.core import Settings

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

#from llama_index.embeddings.ollama import OllamaEmbedding
#from llama_index.llms.ollama import Ollama
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding


from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage

In [3]:
print("LLamaIndex:", llama_index.core.__version__)
print("Pydantic:", pydantic.VERSION)
print("OpenAI:", openai.__version__)

LLamaIndex: 0.14.12
Pydantic: 2.12.5
OpenAI: 2.15.0


#### Test model knowledge without RAG

In [17]:
client = openai.OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_API_KEY"]
)
#model="gpt-4o"
model="gpt-4o-mini"

prompt = f"""Lauder iskola - mit jelent a NYIT?"""

messages=[#{"role": "system", "content": "You are a grumpy and skeptic teacher."},
          #{"role": "system", "content": "Explain me as I were five."},
          {"role": "user", "content": prompt},
          ]

response = client.chat.completions.create(model=model,
                                          messages=messages,
                                          temperature=0,
                                          max_tokens=800,
                                          top_p=1,
                                          frequency_penalty=0,
                                          presence_penalty=0,
                                          stop=None,
                                          )

print(response.choices[0].message.content)

2026-01-16 10:14:56,790 - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


A Lauder Iskola a Budapesti Corvinus Egyetem keretein belül működő intézmény, amely a nemzetközi üzleti és gazdasági képzésekre specializálódott. A NYIT (New York Institute of Technology) egy amerikai egyetem, amelynek programjai és diplomái nemzetközi szinten is elismertek. A Lauder Iskola és a NYIT közötti együttműködés lehetővé teszi a hallgatók számára, hogy a két intézmény által kínált képzéseken részt vegyenek, és így nemzetközi tapasztalatokat szerezzenek. 

Ha további részletekre van szükséged a Lauder Iskoláról vagy a NYIT-ról, szívesen segítek!


#### Defining Models

For using [Ollama Models](https://ollama.com/search), check which ones are installed in your local machine

In [4]:
#model="gpt-4o"
model="gpt-4o-mini"
Settings.llm = OpenAI(temperature=0, 
                      model=model, 
                      #max_tokens=512
                      api_key=os.environ["GITHUB_API_KEY"],
                      api_base="https://models.inference.ai.azure.com",
                     )


Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=os.environ["GITHUB_API_KEY"],
    api_base="https://models.inference.ai.azure.com",
)


#Settings.llm = Ollama(model="llama3.2:latest", request_timeout=300.0)
#Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

#### Defining Folders

In [5]:
print(f"Current dir: {os.getcwd()}")
DOCS_DIR = "../../Data/txt/"
if not os.path.exists(DOCS_DIR):
  os.mkdir(DOCS_DIR)
docs = os.listdir(DOCS_DIR)
docs = [d for d in docs]
docs.sort()
print(f"Files in {DOCS_DIR}")
for doc in docs:
    print(doc)

Current dir: e:\Mega Sync\VS\lauder_ai_lab\GenAI4Humanists\GenAI4Humanists\Notebooks\LlamaIndex
Files in ../../Data/txt/
Pride_Prejudice.txt
RomeoJuliet.txt
WarrenCommissionReport.txt
kafka_metamorphosis.txt
nyc_text.txt
paul_graham_essay.txt
state_of_the_union.txt


In [18]:
documents = SimpleDirectoryReader(input_files=[f"{DOCS_DIR}lauder.txt"]).load_data()
documents

[Document(id_='340cb151-8c69-4bd6-bae0-911b1952e107', embedding=None, metadata={'file_path': '..\\..\\Data\\txt\\lauder.txt', 'file_name': 'lauder.txt', 'file_type': 'text/plain', 'file_size': 3427, 'creation_date': '2026-01-16', 'last_modified_date': '2026-01-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# Filozófiánk\r\n\r\nIskolánk elsődleges célkitűzése a hatékony értékközvetítés, amelyben központi szerepe van a zsidó hagyományok megismertetésének. Célunk, hogy a hozzánk csatlakozó fiatalok magasan képzett, felelősen cselekvő, innovatív tagjaivá váljanak a zsidó közösségnek és a magyar társadalomnak.\r\n\r\nTanítványaink számár

In [26]:
from pprint import pprint

pprint(documents[0].model_dump())


{'audio_resource': None,
 'class_name': 'Document',
 'embedding': None,
 'excluded_embed_metadata_keys': ['file_name',
                                  'file_type',
                                  'file_size',
                                  'creation_date',
                                  'last_modified_date',
                                  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
                                'file_type',
                                'file_size',
                                'creation_date',
                                'last_modified_date',
                                'last_accessed_date'],
 'id_': '340cb151-8c69-4bd6-bae0-911b1952e107',
 'image_resource': None,
 'metadata': {'creation_date': '2026-01-16',
              'file_name': 'lauder.txt',
              'file_path': '..\\..\\Data\\txt\\lauder.txt',
              'file_size': 3427,
              'file_type': 'text/plain',
              'last_modified_date': '

In [29]:
index = VectorStoreIndex.from_documents(documents)

2026-01-16 10:28:37,620 - INFO - HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"


In [34]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import VectorStoreIndex

parser = MarkdownNodeParser()

index = VectorStoreIndex.from_documents(
    documents,
    transformations=[parser],
)



2026-01-16 10:32:55,196 - INFO - HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"


In [35]:
docstore = index.storage_context.docstore
nodes = list(docstore.docs.values())
print(len(nodes))
pprint([n.model_dump() for n in nodes])


3
[{'class_name': 'TextNode',
  'embedding': None,
  'end_char_idx': 796,
  'excluded_embed_metadata_keys': ['file_name',
                                   'file_type',
                                   'file_size',
                                   'creation_date',
                                   'last_modified_date',
                                   'last_accessed_date'],
  'excluded_llm_metadata_keys': ['file_name',
                                 'file_type',
                                 'file_size',
                                 'creation_date',
                                 'last_modified_date',
                                 'last_accessed_date'],
  'id_': 'b5edff7b-7fe1-4a2f-9761-b1d41e6cd243',
  'metadata': {'creation_date': '2026-01-16',
               'file_name': 'lauder.txt',
               'file_path': '..\\..\\Data\\txt\\lauder.txt',
               'file_size': 3427,
               'file_type': 'text/plain',
               'header_path': '/',
       

In [41]:
query_engine = index.as_query_engine(similarity_top_k=3)
response = query_engine.query(prompt)
print(response)

2026-01-16 10:36:44,240 - INFO - HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
2026-01-16 10:36:46,143 - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


A NYIT a nyelvi - informatikai tanévet jelenti, amely a 8. év után következik, és lehetőséget ad arra, hogy a diákok 19 vagy 20 évesen érettségizzenek.


In [42]:
[node.text[0:50] for node in response.source_nodes]

['# Tagozataink\r\n\r\nIntézményünkben óvoda, általános ',
 '# Filozófiánk\r\n\r\nIskolánk elsődleges célkitűzése a',
 '# Alapelveink\r\n\r\nNyitottság — Iskolánk nyitott min']

#### Persist

In [9]:
INDEX_DIR = "../../Index/VectorStoreIndex/"
if not os.path.exists(INDEX_DIR):
  os.mkdir(INDEX_DIR)
index.storage_context.persist(INDEX_DIR)

In [10]:
if not os.path.exists(INDEX_DIR):
    documents = SimpleDirectoryReader(DOCS_DIR).load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=INDEX_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_DIR)
    index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()
response = query_engine.query("What is the document about?")
print(response)

2026-01-16 10:07:20,222 - INFO - Loading all indices.
2026-01-16 10:07:20,465 - INFO - HTTP Request: POST https://models.inference.ai.azure.com/embeddings "HTTP/1.1 200 OK"
2026-01-16 10:07:22,804 - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


The document appears to be a passage from a narrative that explores the dynamics of the Samsa family, particularly focusing on their interactions and emotional states following a significant event. It highlights their decision to take a break from their usual routines, their reflections on work and future prospects, and the evolving relationship between the family members, especially regarding Grete's development and the family's aspirations for her. The atmosphere shifts from tension to a sense of hope as they contemplate moving to a new home and the potential for Grete's future.
